In [1]:
import cv2
import mediapipe as mp 
import joblib
import numpy as np 
from PIL import ImageFont, ImageDraw, Image
import arabic_reshaper
from bidi.algorithm import get_display

In [2]:
string =''
out=''

In [3]:
symbol = {'ع':"ain",
        'ال':"al",
        'ا':"aleff",
        'ب':"bb",
        'د':"dal",
        'ظ':"dha",
        'ض':"dhad",
        'ف':"fa",
        'ق':"gaaf",
        'غ':"ghain",
        'ه':"ha",
        'ح':"haa",
        'ج':"jeem",
        'ك':"kaaf",
        'خ':"khaa",
        'لا':"la",
        'ل':"laam",
        'م':"meem",
        'ن':"nun",
        'ر':"ra",
        'ص':"saad",
        'س':"seen",
        'ش':"sheen",
        'ط':"ta",
        'ت':"taa",
        'ث':"thaa",
        'ذ':"thal",
        'ة':"toot",
        'و':"waw",
        'ئ':"ya",
        'ي':"yaa",
        'ز':"zay"
        }
def getsymbol(s) : 
    for i , j in symbol.items() : 
        if s == j : 
            return i    

In [4]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils


# For webcam input:
hands = mp_hands.Hands(max_num_hands=1,static_image_mode=False, min_detection_confidence=0.7)

cap = cv2.VideoCapture(0)

def data_clean(landmark):
    data = landmark[0]
    data = str(data)
    data = data.strip().split('\n')
  
    garbage = ['landmark {', '}']
  
    landmarks = []
   
    for i in data:
        if i not in garbage:
            landmarks.append(i)

    clean = []

    for i in landmarks:
        i = i.strip()
        clean.append(i[2:])
        
    return([clean])

In [5]:
while cap.isOpened():
  success, image = cap.read()
  image = cv2.flip(image, 1)
  
  if not success:
        break

  # Flip the image horizontally for a later selfie-view display, and convert
  # the BGR image to RGB.
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  # To improve performance, optionally mark the image as not writeable to
  # pass by reference.
  image.flags.writeable = False
  results = hands.process(image)

  # Draw the hand annotations on the image.
  image.flags.writeable = True

  image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

  if results.multi_hand_landmarks:
    for hand_landmarks in results.multi_hand_landmarks:
      mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cleaned_landmark = data_clean(results.multi_hand_landmarks)

    if cleaned_landmark:
      clf = joblib.load('ARL_Model.pkl')
      y_pred = clf.predict(cleaned_landmark)   
################################################################
      #print the predicted in arabic
      out=getsymbol(str(y_pred[0]))    
      reshaped_texta = arabic_reshaper.reshape(out)
      AR_String = get_display(reshaped_texta)
      fontpath = "arial.ttf"
      fontAR = ImageFont.truetype(fontpath, 62)
      img_pil = Image.fromarray(image)
      draw = ImageDraw.Draw(img_pil)
      draw.text((22,100),AR_String, font = fontAR, fill=(0,0,255) )    
      image = np.array(img_pil)
##################################################################
        

##################################################################
  #print the text in arabic
  reshaped_text = arabic_reshaper.reshape(string)
  bidi_text = get_display(reshaped_text) 
  fontpath = "arial.ttf" 
  fontA = ImageFont.truetype(fontpath, 36)
  img_pil = Image.fromarray(image)
  draw = ImageDraw.Draw(img_pil)
  draw.text((22,44),bidi_text, font = fontA, fill=(0,255,0) )
  image = np.array(img_pil)
################################################################
  #print the inputs on screen
  font = cv2.FONT_HERSHEY_SIMPLEX
  cv2.putText(image,"Word:", (22,34), font, 1, (0,255,0), 2, cv2.LINE_AA)
  cv2.putText(image,"S: Add  D: Delete  Esc: Quit", (22,470), font, 1, (255,100,100), 2, cv2.LINE_AA)  
  cv2.imshow('MediaPipe Hands', image) 
  
  
  key = cv2.waitKey(1)  
    
  if key== ord('s'):     
    string = string+""+out  #add the predicted letter letter
    
  if key== ord('d'):
    string=string[:-1]  #delete last letter
        
  if key== 32:  #Space == 32
    string = string+" ";  #add space      
        
  if key== 27: #Escape == 27
    break  # close the application

hands.close()
cap.release()
cv2.destroyAllWindows()
cv2.destroyAllWindows()

In [6]:
print(string)

عاصم
